In [1]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
# Create Complete Daily CSV

df = pd.read_csv('/content/gdrive/MyDrive/rainfall-forecast/station_data/Final results_csv/station_list_final_v2.csv')

daterange = pd.date_range('2016-01-01', '2020-12-31')

date_list = daterange.strftime('%Y-%m-%d').to_list()

df_full_date = pd.DataFrame(date_list, columns=['YEARMODA'])
df_full_date['YEARMODA'] = pd.to_datetime(df_full_date['YEARMODA'])
df_full_date = df_full_date.set_index('YEARMODA')
df_full_date['val'] = np.nan

df_data_complete_date = pd.DataFrame(index=date_list)

for station in range(len(df['STNID'])):
  station_id = df['STNID'].iloc[station]
  station_country = df['ISO3C'].iloc[station]
  station_src = df['SOURCE'].iloc[station]

  if station_src == "TAHMO":
    print("TAHMO - ", station_country)
    
    dir = '/content/gdrive/MyDrive/rainfall-forecast/station_data/TAHMO/TAHMO_daily_station_data.csv'
    station_df = pd.read_csv(dir)

    station_df = station_df[['Unnamed: 0',station_id]]

    station_df['Unnamed: 0'] = pd.to_datetime(station_df['Unnamed: 0'])
    station_df = station_df.drop(station_df[(station_df['Unnamed: 0'] < '2016-01-01') | (station_df['Unnamed: 0'] > '2020-12-31')].index)

    station_df = station_df[station_df[station_id].notna()]

    df_data_date = station_df
    df_data_date = df_data_date.set_index('Unnamed: 0')
    df_data_date.columns = ['val']

    missing_date = df_full_date.index.difference(df_data_date.index)

    df_data_date = pd.concat([df_data_date,df_full_date.loc[missing_date,:]])
    df_data_date = df_data_date.sort_index()
    df_data_date.columns = [station_id]

    df_data_complete_date = df_data_complete_date.join(df_data_date)

  elif station_src == "GSOD":
    print("GSOD")
    dir = '/content/gdrive/MyDrive/rainfall-forecast/station_data/GSOD/' + station_country + "/" + station_id + ".csv"
    station_df = pd.read_csv(dir)

    station_df['YEARMODA'] = pd.to_datetime(station_df['YEARMODA'])
    station_df = station_df.drop(station_df[(station_df['YEARMODA'] < '2016-01-01') | (station_df['YEARMODA'] > '2020-12-31')].index)

    station_df = station_df[station_df['PRCP'].notna()]

    df_data_date = station_df[['YEARMODA','PRCP']]
    df_data_date = df_data_date.set_index('YEARMODA')
    df_data_date.columns = ['val']

    missing_date = df_full_date.index.difference(df_data_date.index)

    df_data_date = pd.concat([df_data_date,df_full_date.loc[missing_date,:]])
    df_data_date = df_data_date.sort_index()
    df_data_date.columns = [station_id]

    df_data_complete_date = df_data_complete_date.join(df_data_date)

  elif station_src == "WEATHER_IMPACT":
    print("Burundi Bob")
    dir = '/content/gdrive/MyDrive/rainfall-forecast/station_data/Burundi_Bob/burundi_bob_obs.csv'
    station_df = pd.read_csv(dir)

    station_df = station_df[['Date',station_id]]

    station_df['Date'] = pd.to_datetime(station_df['Date'])
    station_df = station_df.drop(station_df[(station_df.Date < '2016-01-01') | (station_df.Date > '2020-12-31')].index)

    station_df = station_df[station_df[station_id].notna()]

    df_data_date = station_df
    df_data_date = df_data_date.set_index('Date')
    df_data_date.columns = ['val']

    missing_date = df_full_date.index.difference(df_data_date.index)

    df_data_date = pd.concat([df_data_date,df_full_date.loc[missing_date,:]])
    df_data_date = df_data_date.sort_index()
    df_data_date.columns = [station_id]

    df_data_complete_date = df_data_complete_date.join(df_data_date)

df_data_complete_date = df_data_complete_date.round(1)
df_data_complete_date.to_csv("/content/gdrive/MyDrive/rainfall-forecast/verification_validation/final_station_data_daily_v2.csv")

TAHMO -  KEN
TAHMO -  KEN
TAHMO -  KEN
TAHMO -  KEN
TAHMO -  KEN
GSOD
GSOD
GSOD
GSOD
GSOD
GSOD
GSOD
GSOD
GSOD
GSOD
GSOD
GSOD
GSOD
GSOD
TAHMO -  KEN
TAHMO -  KEN
GSOD
GSOD
TAHMO -  KEN
GSOD
TAHMO -  KEN
TAHMO -  KEN
TAHMO -  KEN
GSOD
GSOD
GSOD
GSOD
GSOD
TAHMO -  KEN
GSOD
TAHMO -  KEN
GSOD
GSOD
TAHMO -  UGA
TAHMO -  KEN
GSOD
TAHMO -  KEN
GSOD
GSOD
TAHMO -  KEN
TAHMO -  RWA
TAHMO -  KEN
TAHMO -  KEN
TAHMO -  KEN
TAHMO -  KEN
TAHMO -  KEN
GSOD
TAHMO -  KEN
TAHMO -  KEN
TAHMO -  KEN
TAHMO -  KEN
GSOD
TAHMO -  KEN
GSOD
GSOD
TAHMO -  KEN
TAHMO -  KEN
TAHMO -  KEN
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burundi Bob
Burun

In [7]:
# Create Monthly Aggregate

df_daily = pd.read_csv('/content/gdrive/MyDrive/rainfall-forecast/verification_validation/final_station_data_daily_v2.csv')
df_daily['Unnamed: 0'] = pd.to_datetime(df_daily['Unnamed: 0'])
df_daily = df_daily.set_index('Unnamed: 0')

df_monthly = df_daily.groupby(pd.Grouper(freq='M')).sum()
df_monthly.to_csv("/content/gdrive/MyDrive/rainfall-forecast/verification_validation/final_station_data_monthly_v2.csv")